In [1]:
import scipy.io
import torch
import numpy as np
import torch.nn as nn
import torch.utils.data as Data
import matplotlib.pyplot as plt
import torch.nn.functional as F
#from tensorboardX import SummaryWriter
from sklearn.metrics import roc_auc_score,roc_curve,auc,average_precision_score,precision_recall_curve
torch.manual_seed(1337)
np.random.seed(1337)
torch.cuda.manual_seed(1337)
torch.backends.cudnn.benchmark=True

# when loading the pred file directly

In [ ]:
print('starting loading the data')
np_test_data = scipy.io.loadmat('test.mat')
testY_data = torch.FloatTensor(np_test_data['testdata'])

In [2]:
BATCH_SIZE = 100

print('starting loading the data')
np_test_data = scipy.io.loadmat('test.mat')
testX_data = torch.FloatTensor(np_test_data['testxdata'])
testY_data = torch.FloatTensor(np_test_data['testdata'])

test_loader = Data.DataLoader(
   dataset=Data.TensorDataset(testX_data, testY_data),
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
   drop_last=False,
)

starting loading the data


In [3]:
print('compling the network')

class DanQ(nn.Module):
    def __init__(self, ):
        super(DanQ, self).__init__()
        self.Conv1 = nn.Conv1d(in_channels=4, out_channels=320, kernel_size=13)
        #nn.init.uniform_(self.Conv1.weight, -0.05, 0.05)
        self.Maxpool = nn.MaxPool1d(kernel_size=13, stride=6)
        self.Drop1 = nn.Dropout(p=0.2)
        self.BiLSTM = nn.LSTM(input_size=320, hidden_size=320, num_layers=2,
                                 batch_first=True,
                                 dropout=0.5,
                                 bidirectional=True)
        self.Linear1 = nn.Linear(163*640, 925)
        self.Linear2 = nn.Linear(925, 919)


    def forward(self, input):
        x = self.Conv1(input)
        x = F.relu(x)
        x = self.Maxpool(x)
        x = self.Drop1(x)
        x_x = torch.transpose(x, 1, 2)
        x, (h_n,h_c) = self.BiLSTM(x_x)
        #x, h_n = self.BiGRU(x_x)
        x = x.contiguous().view(-1, 163*640)
        x = self.Linear1(x)
        x = F.relu(x)
        x = self.Linear2(x)
        #x = torch.sigmoid(x)
        return x

compling the network


In [4]:
danq = DanQ()
danq.load_state_dict(torch.load('model/model0512/danq_net_params_4.pkl'))
danq.cuda()
loss_func = nn.BCEWithLogitsLoss()

In [5]:
print('starting testing')
# training
pred_y = np.zeros([455024, 919])
i=0;j = 0
test_losses = []
danq.eval()
for step, (seq, label) in enumerate(test_loader):
    #print(step)
    seq = seq.cuda()
    label = label.cuda()

    test_output = danq(seq)
    cross_loss = loss_func(test_output, label)
    test_losses.append(cross_loss.item())
    
    test_output = torch.sigmoid(test_output.cpu().data)     

    if(step<4550):
        for i, j in zip(range(step*100, (step+1)*100),range(0, 100)):
            pred_y[i, :] = test_output.numpy()[j, :]
    else:
        for i,j in zip(range(455000,455024),range(0,24)):
            pred_y[i, :] = test_output.numpy()[j, :]
        #print(test_output.numpy())
        
    
test_loss = np.average(test_losses)
print_msg = (f'test_loss: {test_loss:.5f}')  
print(print_msg)    
#np.save('0522pred.npy',pred_y)

starting testing
test_loss: 0.05987


In [ ]:
pred_y = np.load('pred/0419pred.npy')

In [7]:
aucs_file = open('aucs_pyDanQ_13bp.txt', 'w')
aucs_file.write('pyDanQ AU ROC\tpyDanQ AU PR')
for i in range(0,125):
    aucs_file.write('%.5f\t%.5f' % (roc_auc_score(testY_data.data[:, i], pred_y[:, i]),average_precision_score(testY_data.data[:, i], pred_y[:, i])))
    aucs_file.write('\n')
for i in range(125,598):
    aucs_file.write('%.5f\t%.5f' % (roc_auc_score(testY_data.data[:, i], pred_y[:, i]),average_precision_score(testY_data.data[:, i], pred_y[:, i])))
    aucs_file.write('\n')
for i in range(599,815):
    aucs_file.write('%.5f\t%.5f' % (roc_auc_score(testY_data.data[:, i], pred_y[:, i]),average_precision_score(testY_data.data[:, i], pred_y[:, i])))
    aucs_file.write('\n')
for i in range(815,919):
    aucs_file.write('%.5f\t%.5f' % (roc_auc_score(testY_data.data[:, i], pred_y[:, i]),average_precision_score(testY_data.data[:, i], pred_y[:, i])))
    aucs_file.write('\n')
aucs_file.close()

In [ ]:
roc_auc_score(testY_data.data[:, 598], pred_y[:, 598])

In [ ]:
for i in (211,277):
    precision, recall, thresholds = precision_recall_curve(testY_data.data[:, i], pred_y[:, i])
    ap = average_precision_score(testY_data.data[:, i], pred_y[:, i])
    plt.plot(recall, precision, lw=1, label='%s(AP = %0.4f)' % (str(i),ap))
    plt.plot(recall, precision, lw=1)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.xlim([0, 1.05])
    plt.ylim([0, 1.05])
    
plt.title('pr_curve')
plt.legend()
plt.show()

In [ ]:
for i in (211,277):
    fpr, tpr, thresholds = roc_curve(testY_data.data[:, i], pred_y[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=1, label='%s(AUC = %0.4f)' % (str(i),roc_auc))
    plt.plot(fpr, tpr, lw=1)
    plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6))
    plt.xlim([0, 1.05])
    plt.ylim([0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    
plt.title('roc_curve')
plt.legend()
plt.show()

In [ ]:
ap_=[]
for i in range(0,125):
    ap_.append(average_precision_score(testY_data.data[:, i], pred_y[:, i]))
ap = np.average(ap_)
print_msg = (f'ap_DNase: {ap:.5f}')  
print(print_msg)
ap_=[]
for i in range(125,598):
    ap_.append(average_precision_score(testY_data.data[:, i], pred_y[:, i]))
for i in range(599,815):
    ap_.append(average_precision_score(testY_data.data[:, i], pred_y[:, i]))
ap = np.average(ap_)
print_msg = (f'ap_TFBinding: {ap:.5f}')  
print(print_msg)
ap_=[]
for i in range(815,919):
    ap_.append(average_precision_score(testY_data.data[:, i], pred_y[:, i]))
ap = np.average(ap_)
print_msg = (f'ap_Histone: {ap:.5f}')  
print(print_msg)

In [ ]:
auc_=[]
for i in range(0,125):
    auc_.append(roc_auc_score(testY_data.data[:, i], pred_y[:, i]))
ap = np.average(auc_)
print_msg = (f'auc_DNase: {ap:.5f}')  
print(print_msg)
auc_=[]
for i in range(125,598):
    auc_.append(roc_auc_score(testY_data.data[:, i], pred_y[:, i]))
for i in range(599,815):
    auc_.append(roc_auc_score(testY_data.data[:, i], pred_y[:, i]))
ap = np.average(auc_)
print_msg = (f'auc_TFBinding: {ap:.5f}')  
print(print_msg)
auc_=[]
for i in range(815,919):
    auc_.append(roc_auc_score(testY_data.data[:, i], pred_y[:, i]))
ap = np.average(auc_)
print_msg = (f'auc_Histone: {ap:.5f}')  
print(print_msg)

In [ ]:
num=910
print(roc_auc_score(testY_data.data[:, num], pred_y[:, num]))
print(average_precision_score(testY_data.data[:, num], pred_y[:, num]))

In [ ]:
print('printing the pr_curve_125_DNase')
for i in range(0,125):
    precision, recall, thresholds = precision_recall_curve(testY_data.data[:, i], pred_y[:, i])
    #ap = average_precision_score(testY_data.data[:, i], pred_y[:, i])
    #plt.plot(recall, precision, lw=1, label='%s(AP = %0.4f)' % (str(i),ap))
    plt.plot(recall, precision, lw=1)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.xlim([0, 1.05])
    plt.ylim([0, 1.05])
    
plt.title('pr_curve_125_DNase')
plt.savefig('pr_curve_125_DNase.svg')
plt.show()


In [ ]:
print('printing the pr_curve_690_TFbinding')
for i in range(125,815):
    precision, recall, thresholds = precision_recall_curve(testY_data.data[:, i], pred_y[:, i])
    #ap = average_precision_score(testY_data.data[:, i], pred_y[:, i])
    plt.plot(recall, precision, lw=1)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.xlim([0, 1.05])
    plt.ylim([0, 1.05])
    
plt.title('pr_curve_690_TFbinding')
plt.savefig('pr_curve_690_TFbinding.png')
plt.show()

In [ ]:


print('printing the pr_curve_104_Histone')
for i in range(815,919):
    precision, recall, thresholds = precision_recall_curve(testY_data.data[:, i], pred_y[:, i])
    #ap = average_precision_score(testY_data.data[:, i], pred_y[:, i])
    plt.plot(recall, precision, lw=1)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.xlim([0, 1.05])
    plt.ylim([0, 1.05])
    
plt.title('pr_curve_104_Histone')
plt.savefig('pr_curve_104_Histone.svg')
plt.show()


In [ ]:
print('printing the roc_curve_125_DNase')
for i in range(0,125):
    fpr, tpr, thresholds = roc_curve(testY_data.data[:, i], pred_y[:, i])
    roc_auc = auc(fpr, tpr)
    #plt.plot(fpr, tpr, lw=1, label='%s(AUC = %0.4f)' % (str(i),roc_auc))
    plt.plot(fpr, tpr, lw=1)
    plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6))
    plt.xlim([0, 1.05])
    plt.ylim([0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    
plt.title('roc_curve_125_DNase')
plt.savefig('roc_curve_125_DNase.svg')
plt.show()

print('printing the roc_curve_690_TFbinding')
for i in range(125,815):
    fpr, tpr, thresholds = roc_curve(testY_data.data[:, i], pred_y[:, i])
    #roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=1)
    plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6))
    plt.xlim([0, 1.05])
    plt.ylim([0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    
plt.title('roc_curve_690_TFbinding')
plt.savefig('roc_curve_690_TFbinding.svg')
plt.show()

print('printing the roc_curve_104_Histone')
for i in range(815,919):
    fpr, tpr, thresholds = roc_curve(testY_data.data[:, i], pred_y[:, i])
    #roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=1)
    plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6))
    plt.xlim([0, 1.05])
    plt.ylim([0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    
plt.title('roc_curve_104_Histone')
plt.savefig('roc_curve_104_Histone.svg')
plt.show()

In [ ]:
import scipy.io
import numpy as np
from sklearn.metrics import roc_auc_score,roc_curve,auc,average_precision_score,precision_recall_curve

In [ ]:
print('starting loading the data')
np_test_data = scipy.io.loadmat('test.mat')
testY_data = np_test_data['testdata']
pred_y = np.load('0419pred.npy')

In [ ]:
print('125_DNase')
auc_5=0
auc_6=0
auc_7=0
auc_8=0
auc_9=0
for i in range(0,125):
    if roc_auc_score(testY_data[:, i], pred_y[:, i])>0.9:
        auc_9 +=1
    elif roc_auc_score(testY_data[:, i], pred_y[:, i])>0.8 and roc_auc_score(testY_data[:, i], pred_y[:, i])<0.9:
        auc_8 +=1
    elif roc_auc_score(testY_data[:, i], pred_y[:, i])>0.7 and roc_auc_score(testY_data[:, i], pred_y[:, i])<0.8:
        auc_7 +=1
    elif roc_auc_score(testY_data[:, i], pred_y[:, i])>0.6 and roc_auc_score(testY_data[:, i], pred_y[:, i])<0.7:
        auc_6 +=1
    elif roc_auc_score(testY_data[:, i], pred_y[:, i])>0.5 and roc_auc_score(testY_data[:, i], pred_y[:, i])<0.6:
        auc_5 +=1
print_msg = (f'auc_9: {auc_9:.3f}'+'\n'
             f'auc_8: {auc_8:.3f}'+'\n'
             f'auc_7: {auc_7:.3f}'+'\n'
             f'auc_6: {auc_6:.3f}'+'\n'
             f'auc_5: {auc_5:.3f}')  
print(print_msg)

In [ ]:
print('690_TFbinding')
auc_5=0
auc_6=0
auc_7=0
auc_8=0
auc_9=0
for i in range(125,598):
    if roc_auc_score(testY_data[:, i], pred_y[:, i])>0.9:
        auc_9 +=1
    elif roc_auc_score(testY_data[:, i], pred_y[:, i])>0.8 and roc_auc_score(testY_data[:, i], pred_y[:, i])<0.9:
        auc_8 +=1
    elif roc_auc_score(testY_data[:, i], pred_y[:, i])>0.7 and roc_auc_score(testY_data[:, i], pred_y[:, i])<0.8:
        auc_7 +=1
    elif roc_auc_score(testY_data[:, i], pred_y[:, i])>0.6 and roc_auc_score(testY_data[:, i], pred_y[:, i])<0.7:
        auc_6 +=1
    elif roc_auc_score(testY_data[:, i], pred_y[:, i])>0.5 and roc_auc_score(testY_data[:, i], pred_y[:, i])<0.6:
        auc_5 +=1
for i in range(599,815):
    if roc_auc_score(testY_data[:, i], pred_y[:, i])>0.9:
        auc_9 +=1
    elif roc_auc_score(testY_data[:, i], pred_y[:, i])>0.8 and roc_auc_score(testY_data[:, i], pred_y[:, i])<0.9:
        auc_8 +=1
    elif roc_auc_score(testY_data[:, i], pred_y[:, i])>0.7 and roc_auc_score(testY_data[:, i], pred_y[:, i])<0.8:
        auc_7 +=1
    elif roc_auc_score(testY_data[:, i], pred_y[:, i])>0.6 and roc_auc_score(testY_data[:, i], pred_y[:, i])<0.7:
        auc_6 +=1
    elif roc_auc_score(testY_data[:, i], pred_y[:, i])>0.5 and roc_auc_score(testY_data[:, i], pred_y[:, i])<0.6:
        auc_5 +=1
print_msg = (f'auc_9: {auc_9:.3f}'+'\n'
             f'auc_8: {auc_8:.3f}'+'\n'
             f'auc_7: {auc_7:.3f}'+'\n'
             f'auc_6: {auc_6:.3f}'+'\n'
             f'auc_5: {auc_5:.3f}')  
print(print_msg)

In [ ]:
print('104_Histone')
auc_5=0
auc_6=0
auc_7=0
auc_8=0
auc_9=0
for i in range(815,919):
    if roc_auc_score(testY_data[:, i], pred_y[:, i])>0.9:
        auc_9 +=1
    elif roc_auc_score(testY_data[:, i], pred_y[:, i])>0.8 and roc_auc_score(testY_data[:, i], pred_y[:, i])<0.9:
        auc_8 +=1
    elif roc_auc_score(testY_data[:, i], pred_y[:, i])>0.7 and roc_auc_score(testY_data[:, i], pred_y[:, i])<0.8:
        auc_7 +=1
    elif roc_auc_score(testY_data[:, i], pred_y[:, i])>0.6 and roc_auc_score(testY_data[:, i], pred_y[:, i])<0.7:
        auc_6 +=1
    elif roc_auc_score(testY_data[:, i], pred_y[:, i])>0.5 and roc_auc_score(testY_data[:, i], pred_y[:, i])<0.6:
        auc_5 +=1
print_msg = (f'auc_9: {auc_9:.3f}'+'\n'
             f'auc_8: {auc_8:.3f}'+'\n'
             f'auc_7: {auc_7:.3f}'+'\n'
             f'auc_6: {auc_6:.3f}'+'\n'
             f'auc_5: {auc_5:.3f}')  
print(print_msg)